# Extract accessions from supplementary table 3

In [1]:
import pandas as pd

dfs = pd.read_excel("Supplementary Tables 1-3.xlsx", skiprows=1, sheet_name=None)

before_date = pd.to_datetime("2020-03-01")

In [2]:
table_s3 = (
    dfs["Supplementary Table 3"]
    .rename(
        columns={
            "Accession ID": "accession",
            "Strain name": "strain",
            "Lineage": "lineage",
            "Collection date": "date",
        }
    )
    [["accession", "strain", "lineage", "date"]]
    .assign(date=lambda x: pd.to_datetime(x["date"]))
    .query("date < @before_date")
)

table_s3

,accession,strain,lineage,date
0,EPI_ISL_402123,Wuhan/IPBCAMS-WH-01/2019,B,2019-12-24
1,EPI_ISL_406798,Wuhan/WH01/2019,B,2019-12-26
2,MN908947.3,Wuhan-Hu-1-/2019-12-26,B,2019-12-26
3,EPI_ISL_529217,Wuhan/IME-WH05/2019,B,2019-12-30
4,EPI_ISL_529216,Wuhan/IME-WH04/2019,B,2019-12-30
...,...,...,...,...
1742,EPI_ISL_413022,Switzerland/ZH-UZH-1000477796/2020,B.1,2020-02-29
1743,EPI_ISL_413021,Switzerland/ZH-UZH-1000477757/2020,B.1,2020-02-29
1744,EPI_ISL_1064059,Italy/VEN-SR-72-B/2020,B,2020-02-29
1745,EPI_ISL_1015045,France/IDF_PSL_141/2020,B.1,2020-02-29


In [3]:
annotations = pd.read_csv("jointWHO_sequences.csv")

annotations

,joint_WHO_China_sample_id,strain,accession,from_market,addtl_annotations
0,S01,hCoV-19/Wuhan/IPBCAMS-WH-05/2020,EPI_ISL_403928,True,2019 market sequence
1,S02,hCoV-19/Wuhan/IPBCAMS-WH-01/2019,EPI_ISL_402123,True,2019 market sequence
2,S03,hCoV-19/Wuhan/WH01/2019,EPI_ISL_406798,True,2019 market sequence
3,S04,hCoV-19/Wuhan/WIV02/2019,EPI_ISL_402127,True,2019 market sequence
4,S04,hCoV-19/Wuhan/IME-WH02/2019,EPI_ISL_529214,True,2019 market sequence
5,S04,hCoV-19/Wuhan/HBCDC-HB-02/2019,EPI_ISL_412898,True,2019 market sequence
6,S04,hCoV-19/Wuhan/IVDC-HB-GX02/2019,EPI_ISL_434534,True,2019 market sequence
7,S05,hCoV-19/Wuhan/IPBCAMS-WH-04/2019,EPI_ISL_403929,True,2019 market sequence
8,S05,hCoV-19/Wuhan/IVDC-HB-04/2020,EPI_ISL_402120,True,2019 market sequence
9,S06,Wuhan-Hu-1,MN908947.3,True,2019 market sequence


In [4]:
merged = table_s3.merge(
    annotations[["accession", "joint_WHO_China_sample_id", "addtl_annotations"]],
    on="accession",
    how="left",
    validate="one_to_one",
).assign(addtl_annotations=lambda x: x["addtl_annotations"].fillna("other"))

In [5]:
merged.to_csv("all_accessions.csv", index=False)

with open("gisaid_accessions.csv", "w") as f:
    f.write(
        ", ".join(merged.query("accession.str.startswith('EPI_')")["accession"].tolist())
    )
    
with open("genbank_accessions.csv", "w") as f:
    f.write(
        ", ".join(merged.query("not accession.str.startswith('EPI_')")["accession"].tolist())
    )    